In [52]:

from PIL import Image


In [60]:
import requests
import json
server_url='https://bf44-52-66-155-210.ngrok-free.app'
def inpaint_image(image_path, bounding_boxes, output_path='inpainted_image.png'):
    """
    Sends a request to the inpainting API endpoint and saves the inpainted image.

    Parameters:
    - image_path: str, path to the image file to be inpainted.
    - bounding_boxes: list of tuples, each tuple containing (x1, y1, x2, y2) coordinates.
    - output_path: str, path where the inpainted image will be saved (default is 'inpainted_image.png').

    Returns:
    - success: bool, True if the image was successfully inpainted and saved, False otherwise.
    - error: str or None, error message if the request fails.
    """
    url = server_url+'/inpaint'

    try:
        with open(image_path, 'rb') as image_file:
            files = {
                'image': image_file
            }
            data = {
                'bboxes': json.dumps(bounding_boxes)  # Convert bounding boxes to a JSON string
            }

            response = requests.post(url, files=files, data=data)

            if response.status_code == 200:
                # Save the inpainted image to the specified output path
                with open(output_path, 'wb') as out_file:
                    out_file.write(response.content)
                return True, None  # Indicate success and no error
            else:
                return False, response.json().get('error', 'Unknown error occurred.')
    except Exception as e:
        return False, str(e)

In [53]:
def sam_method(image_path, text_input):
    url = server_url+'/process_image'  # URL of the Flask endpoint
    # Prepare the files and data
    
    files = {'image': open(image_path, 'rb')}  # Open the image file in binary mode
    data = {'text_input': text_input}
    # Send a POST request to the endpoint
    response = requests.post(url, files=files, data=data)
    # Check the response
    response=response.json()
    title_bbox = None
    subtitle_bbox = None  # Placeholder for subtitle (not provided in the data)
    product_bbox=None
    for item in response['results']:
        try:
            label = item['result']['bboxes_labels'][0]
            bbox = item['result']['bboxes'][0]

            if label == text_input.split(',')[0]:
                title_bbox = bbox
            elif label == text_input.split(',')[1]:  # Assuming hashtags could serve as a subtitle
                subtitle_bbox = bbox
            elif label == text_input.split(',')[2]:  # Assuming hashtags could serve as a subtitle
                product_bbox = bbox
        except:
            pass
    return title_bbox,subtitle_bbox,product_bbox



In [124]:
image_path='templates/instagram_pic (3).jpg'
image=Image.open(image_path)

In [130]:
query="title,hashtags,product"

In [131]:
title_box,description_box,product_box=sam_method(image_path=image_path,text_input=query)

In [132]:
inpaint_image(image_path=image_path,bounding_boxes=[title_box,description_box])

(True, None)